- DFT data to break down the wave
- Signal fragmentatioin to overcome large number of samples for training the model
- LSTM model is implemented for each type of brainwave
- 

## Import libraries

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import os, math
import tensorflow as tf
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM

from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

dir = os.getcwd()
path = os.path.join(dir, 'zone_prod_70')
os.chdir(path)
print('Current working directory: {}'.format(os.getcwd()))


Current working directory: c:\Users\ryoii\OneDrive\Documents\work\zone_prod_70


## Read csv files

- _sequences: contains raw EEG data
- _labels: contains labels for the csv files
0 = base, 1 = target

In [5]:
def load_files(path):
    all_data = pd.DataFrame()

    for index, folder in enumerate(path):
        for file in os.listdir(folder):
            if file.endswith('.csv'):
                df = pd.read_csv(folder+'\\'+file, header=0, usecols=['Left Raw', 'Right Raw'])
                df['AVG'] = (df['Left Raw'] + df['Right Raw']) / 2
                df = df[1000:9680]
                if index < 2:
                    df = df.append({'AVG': 0}, ignore_index=True)
                else:
                    df = df.append({'AVG': 1}, ignore_index=True)

                all_data = pd.concat([all_data, df['AVG'].to_frame().T], axis=0, ignore_index=True)

    all_data.to_csv('all_data.csv', index=False)



paths = [os.path.join(os.getcwd(), 'train\\base'), os.path.join(os.getcwd(), 'test\\base'), os.path.join(os.getcwd(), 'train\\base'), os.path.join(os.getcwd(), 'test\\target')]
load_files(paths)

# train_target_sequences = load_files(os.path.join(os.getcwd(), 'train\\target'))
# test_base_sequences = load_files(os.path.join(os.getcwd(), 'test\\base'))
# test_target_sequences = load_files(os.path.join(os.getcwd(), 'test\\target'))

# print('Size of train combined: {}'.format(len(train_base_sequences) + len(train_target_sequences)))
# print('Size of test combined: {}'.format(len(test_base_sequences) + len(test_target_sequences)))

C:\Users\ryoii\AppData\Local\Temp\ipykernel_16556\2017317339.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'AVG': 0}, ignore_index=True)
C:\Users\ryoii\AppData\Local\Temp\ipykernel_16556\2017317339.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'AVG': 0}, ignore_index=True)
C:\Users\ryoii\AppData\Local\Temp\ipykernel_16556\2017317339.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'AVG': 0}, ignore_index=True)
C:\Users\ryoii\AppData\Local\Temp\ipykernel_16556\2017317339.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'AVG': 0}, ignore_index=True)
C:\U

## Preprocessing

Identify minimum series and truncate all other series to that size

In [12]:
def truncate_seq(sequences):
    len_sequences = []

    for seq in (train_sequences + test_sequences):
        len_sequences.append(len(seq))

    min = pd.Series(len_sequences).describe()['min']

    return pad_sequences(sequences, maxlen=1000) #Remove values from the beginning of the sequence, 10 seconds worth of data

truncated_train_sequences = truncate_seq(train_sequences)
truncated_validation_sequences = truncate_seq(validation_sequences)
truncated_test_sequences = truncate_seq(test_sequences)


print('Length of each data: {}'.format(len(truncated_train_sequences[0])))
print('Shape of truncated_train_sequences: {}'.format(truncated_train_sequences.shape))
print('Shape of truncated_validation_sequences: {}'.format(truncated_validation_sequences.shape))
print('Shape of truncated_test_sequences: {}'.format(truncated_test_sequences.shape))


Length of each data: 1000
Shape of truncated_train_sequences: (567, 1000, 2)
Shape of truncated_validation_sequences: (60, 1000, 2)
Shape of truncated_test_sequences: (59, 1000, 2)


- Outliers(value > 3 stds) == 3 std
- Standardize

In [31]:
std = StandardScaler()

def preprocess_seq(sequences):
    sequences = sequences.astype(float)
    for seq in range(sequences.shape[0]):
        df = pd.DataFrame(truncated_train_sequences[seq])

        q_left_upper = df.iloc[:,0].quantile(0.95865)
        q_left_lower = df.iloc[:,0].quantile(0.00135)

        q_right_upper = df.iloc[:,1].quantile(0.95865)
        q_right_lower = df.iloc[:,1].quantile(0.00135)

        df.iloc[:,0] = df.iloc[:,0].apply(lambda x: q_left_upper if x > q_left_upper else x)
        df.iloc[:,0] = df.iloc[:,0].apply(lambda x: q_left_lower if x < q_left_lower else x)
        df.iloc[:,1] = df.iloc[:,1].apply(lambda x: q_right_upper if x > q_right_upper else x)
        df.iloc[:,1] = df.iloc[:,1].apply(lambda x: q_right_lower if x < q_right_lower else x)

        df_scaled = df.copy()
        df_scaled = df = std.fit_transform(df)
        df_scaled = std.transform(df)

        sequences[seq] = df_scaled

        # fig,axes = plt.subplots(2, 1, figsize=(6,10))

        # for i, ax in enumerate(axes.flatten()):
        #     y = df.iloc[:, i].values
        #     ax.plot(y)
        #     axes[i].set_ylabel('Amplitude')
        #     plt.xlabel('Sample Point')
        # plt.show()
    return sequences

preprocessed_train_sequences = preprocess_seq(truncated_train_sequences)
preprocessed_validation_sequences = preprocess_seq(truncated_validation_sequences)
preprocessed_test_sequences = preprocess_seq(truncated_test_sequences)


print('Shape of prepocessed_train_sequences: {}'.format(preprocessed_train_sequences.shape))
print('Shape of preprocessed_validation_sequences: {}'.format(preprocessed_validation_sequences.shape))
print('Shape of preprocessed_test_sequences: {}'.format(preprocessed_test_sequences.shape))


Shape of prepocessed_train_sequences: (567, 1000, 2)
Shape of preprocessed_validation_sequences: (60, 1000, 2)
Shape of preprocessed_test_sequences: (59, 1000, 2)


## Building a Classification model

In [ ]:
model = Sequential()
model.add(LSTM(256, input_shape=(truncated_train_sequences.shape[1], truncated_train_sequences.shape[2])))
model.add(Dense(1, activation='sigmoid'))

model.summary()

In [ ]:
adam = Adam(learning_rate=0.0125)
name = ""
chk = ModelCheckpoint(name, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
history = model.fit(truncated_train_sequences, train_labels, epochs=50, batch_size=128, callbacks=[chk], shuffle=True, validation_data=(truncated_validation_sequences,validation_labels))

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

## Testing accuracy

In [ ]:
model_path = os.path.join(os.getcwd(), "lr-0125_maxlen-6000_epoch-200")
model = load_model(model_path)
test_preds = model.predict(truncated_test_sequences)
classes_pred = np.round(test_preds)
accuracy = accuracy_score(test_labels, classes_pred)
print('Accuracy:', accuracy)